### St Michael Bay

**Station Name:**  at01

**Location:**  St Michael, Alaska

**Archive:**  [EarthScope Consortium](http://www.earthscope.org)

**Ellipsoidal Coordinates:**

- Latitude:  63.4840

- Longitude: -162.0064

- Height: 21.7 m

[Station Page at EarthScope](https://www.unavco.org/instrumentation/networks/status/nota/overview/AT01)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/AT01.sta)

[Google Map Link](https://goo.gl/maps/uWyoNFf4DRjYLmfUA)

<p align="center">
<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/AT01.jpg" width="500">
</p>

## Data Summary

at01 has good visibility over a large azimuthal region and the monument is about a meter taller than normal geodetic monuments which helps improve visibility of the sea surface. At01 observes all constellation signals.

## Imports

In [ ]:
from gnssrefl.subdaily_cl import subdaily
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

# reflecion zone app

now we can use the reflection zone part of the web app:

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/geoid" width="1000" height="600"></iframe>

Next, we could look at the reflection zone app to pick a possible mask.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/rzones" width="1000" height="700"></iframe>

Just looking at this can give us an idea of the possible azimuth mask - Guessing we will not get good retrievals in the northwest quadrant. We will confirm this with the qc plots when we use quicklook.

It's also useful to note that it is about 12 meters above sea level.

### Take a Quick Look at the Data
First examine the spectral characteristics of the SNR data.

First, we download a snr file using rinex2snr. We will pick a year and day - in this case we'll look at day of year 109 for the year 2020. 

In [ ]:
# Download SNR
# we are going to just pick a year and day
# this will generate a SNR file at a sampling rate of 15 seconds.
station = "at01"
year = 2020
doy = 109

rinex2snr(station=station, year=year, doy=doy, archive="unavco", orb="gnss")

Now we can run quicklook, this is what it will look like if we don't change any of the defualts:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

Using the default settings with quicklook we would likely think it is not a useful site, as nearly every retrieval is set as bad.

Let's see what the defaults are set to by adding a question mark to the function and running that cell.

In [ ]:
quicklook?

Now we can see what the defaults are and information about the parameters in the doc string as well. 

So, remembering that the site is 12 meters above sea level and we can see that the default restricts the reflector height region to 0.5 to 6 meters. 

Let's try quicklook again using a reflector height region that includes the water surface. We'll set h1 to 8 and h2 to 15 and we're also going to change the elevation angles to 5 and 13 degrees:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, e1=5, e2=13, h1=8, h2=15)

Now we see good retrievals at azimuths from true north to about 220 degrees (around what we expected from using the reflection zone app) and we also see strong retrievals in the Lomb Scargle periodograms.

This site does have modern GPS signals, as well as Galileo and Glonass signals.

results for L2C:

In [ ]:
values, metrics = quicklook(station, year, doy=doy,
                            e1=5, e2=13, h1=8, h2=15, fr=20)

Now we can look at glonass by setting fr to 101 and Galileo we will set f to 205.

In [ ]:
values_l1, metrics_l1 = quicklook(
    station, year, doy=doy, e1=5, e2=13, h1=8, h2=15, fr=101
)

In [ ]:
values_l2, metrics_l2 = quicklook(
    station, year, doy=doy, e1=5, e2=13, h1=8, h2=15, fr=205
)

### Analyze the Data

Next we will analyze data for a few weeks in the fall of 2020. Our first step would be to make the SNR files.

In [ ]:
rinex2snr?

In [ ]:
rinex2snr(station, year, 230, doy_end=390, archive="unavco", orb="gnss")

The next step is to create the json file that gnssir reads with the values that we have decided on from the quicklook step. In this case we will set the elevation angles and the heights. We'll set the azimuth range that we decided on and are also going to remove the Beidou signals since they are not in rinex2 files.

In [ ]:
make_gnssir_input(
    station,
    e1=5,
    e2=13,
    h1=8,
    h2=17,
    ampl=4,
    frlist=[1, 20, 5, 101, 102, 201, 205, 206, 207],
    azlist2=[20, 220],
)

Next, we will run gnssir to estimate the reflector height (RH) for the two month time period.

In [ ]:
gnssir(station, year, 230, doy_end=290)

We have do have some code to help you look at subdaily files. This will be helpful to look at tides. 

We will set an outlier criteria of 0.36 meters (one sigma 0.12 meters ==> 0.36 meters three sigma) to start with.

This preliminary version of the code removes outliers and makes an effort to compute the RH dot correction.

In [ ]:
subdaily(station="at01", year=2020, doy1=230, doy2=290, plt=True)